# Building a Student Intervention System ( *Supervised Learning*)
<hr>
## Machine Learning Engineer Nanodegree
<hr>
### Tetevi Placide Ekon
September 10<sup>th</sup>, 2016

## Problem Statement
Education at all levels has been undergoing a tremendous amount of change over the last several years. There was a time when the only way to access information was by sifting through books in a library. Today, a student can obtain an encyclopedia's worth of information with a few clicks of a mouse. Companies like Coursera, Udacity, and Khan Academy are democratizing education, while universities are rethinking the traditional approach and incorporating technology to diversify their offerings. As education has grown to rely more and more on technology, there has been an accompanying influx of data, with logs of student activities, grades, and interactions with teachers and fellow students all available in real time.

From the perspective of an educator, this data offers a lot of potential. It can be used to understand how different students learn, which in turn may help identify new ways of predicting success and failure. If these predictions can be made early enough, then effective interventions can be made in a timely manner to ensure the student has the best chance of succeeding. Universities are now even beginning to offer Netflix-like [course recommendations](http://www.datasciencecentral.com/profiles/blogs/the-hastings-effect-how-the-netflix-big-data-approach-is) to their students!

In this project, I seek to model the factors that best predict how likely a student is to pass their high school final exam. My goal is to identify students who might need early intervention. Therefore, this is a supervised classification problem, because I am predicting a binary response: Either the student passes (and does not need an intervention), or the student does not pass (and does require an intervention).


## Exploring the Data
Loading necessary Python libraries and the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [3]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print("Student data read successfully!")

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, we compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [7]:
# Calculate number of students
n_students = len(student_data[0:])

# Calculate number of features
n_features = len(student_data.columns[:-1])

# Calculate passing students
n_passed = len(student_data[student_data.passed == 'yes'])

# Calculate failing students
n_failed = len(student_data[student_data.passed == 'no'])

# Calculate graduation rate
grad_rate = n_passed*100.0/n_students

# Print the results
print ("Total number of students: {}".format(n_students))
print ("Number of features: {}".format(n_features))
print ("Number of students who passed: {}".format(n_passed))
print ("Number of students who failed: {}".format(n_failed))
print ("Graduation rate of the class: {:.2f}%".format(grad_rate))

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identification of feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

The code cell below separates the student data into feature and target columns to see if any features are non-numeric.

In [9]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print ("Feature columns:\n{}".format(feature_cols))
print ("\nTarget column: {}".format(target_col))

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print ("\nFeature values:")
print (X_all.head())

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocessing of Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. The code cell below performs the preprocessing routine discussed in this section.

In [10]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print ("Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns)))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, we implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the `train_test_split` function.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [11]:
from sklearn.cross_validation import train_test_split

# Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=float(num_test)/X_all.shape[0], random_state=0)

# Show the results of the split
print ("Training set has {} samples.".format(X_train.shape[0]))
print ("Testing set has {} samples.".format(X_test.shape[0]))

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, we choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. We first discuss the reasoning behind choosing these three models by considering what we know about the data and each model's strengths and weaknesses. We then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. We finally produce three tables (one for each model) that show the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) 
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Model Application



* **Logistic Regression**  
A real-world application of this method can be predicting if a customer will take a loan from a bank or not.  
    Its strengths are that it performs well even when you have a data where the decision boundary is not parallel to the [`axis`](https://www.quora.com/What-are-the-advantages-of-logistic-regression-over-decision-trees), its low variance makes it less prone to overfitting.  
    Its weaknesses are that it performs poorly when you have more than a single decision boundary.  
    It is a good candidate for this problem because of the probable noisiness in the data given the number of features it contains. There is also the fact that there is a single decision boundary: whether a student needs intervention or not and Logistic Regression Models work very well on these sorts of problems. 
* **K-Nearest Neighbors**   
A real-world application of this method is in handwritten digits recognition.   
Its strengths are that it is simple and that it performs well in classification problems where the decision boundary is very irregular.  
Its weakness is that it is non-generalizing since it only [`remembers`](http://scikit-learn.org/stable/modules/neighbors.html) all of its training data.  
This model is also a good candidate for the problem because of the large number of samples and the low dimensionality in the data.  
* **Decision Trees**  
A real-world application of this method can be identifying the Author of a certain document.       
They are simple to understand and to interpret and can be visualised. They can handle both categorical and numerical values and perform well even if their assumptions are violated by the true model from which the data were [`generated`](http://scikit-learn.org/stable/modules/tree.html).  
Their disadvantages are that their learners can create very complex trees that do not generalize the data well; small variations in the data can result in completely different trees.   
They are a good candidate for this problem because they are easily interpretable and given the large number of features in our dataset, the ability to accurately interprete our model's result can be very valuable.

### Setup
The code cell below initializes three helper functions which can be used for training and testing the three supervised learning models chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [12]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print ("Trained model in {:.4f} seconds".format(end - start))

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print ("Made predictions in {:.4f} seconds.".format(end - start))
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print ("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print ("F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train)))
    print ("F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test)))

### Implementation: Model Performance Metrics
With the predefined functions above, we now import the three supervised learning models chosen previously and run the `train_predict` function for each one. We need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, we should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, we implement the following:
- Import the three supervised learning models discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model used, if provided.
 - **Note:** Here we use the default settings for each model — we will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *We do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  

In [14]:
# Import the three supervised learning models from sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.tree import DecisionTreeClassifier

# Initialize the three models
clf_A = LogisticRegression(random_state = 0)
clf_B = KNeighborsClassifier()
clf_C = DecisionTreeClassifier(random_state = 0)

# Set up the training set sizes
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train[:300]
y_train_300 = y_train[:300]

# Execute the 'train_predict' function for each classifier and each training set size
#train_predict(clf, X_train, y_train, X_test, y_test)

In [15]:
train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)

Training a LogisticRegression using a training set size of 100. . .
Trained model in 0.0074 seconds
Made predictions in 0.0056 seconds.
F1 score for training set: 0.8571.
Made predictions in 0.0020 seconds.
F1 score for test set: 0.7612.


In [16]:
train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)

Training a LogisticRegression using a training set size of 200. . .
Trained model in 0.0146 seconds
Made predictions in 0.0053 seconds.
F1 score for training set: 0.8380.
Made predictions in 0.0039 seconds.
F1 score for test set: 0.7794.


In [17]:
train_predict(clf_A, X_train_300, y_train_300, X_test, y_test)

Training a LogisticRegression using a training set size of 300. . .
Trained model in 0.0196 seconds
Made predictions in 0.0052 seconds.
F1 score for training set: 0.8381.
Made predictions in 0.0037 seconds.
F1 score for test set: 0.7910.


In [18]:
train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)

Training a KNeighborsClassifier using a training set size of 100. . .
Trained model in 0.0035 seconds
Made predictions in 0.0105 seconds.
F1 score for training set: 0.7972.
Made predictions in 0.0073 seconds.
F1 score for test set: 0.7068.


In [19]:
train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)

Training a KNeighborsClassifier using a training set size of 200. . .
Trained model in 0.0138 seconds
Made predictions in 0.0287 seconds.
F1 score for training set: 0.8571.
Made predictions in 0.0225 seconds.
F1 score for test set: 0.7121.


In [20]:
train_predict(clf_B, X_train_300, y_train_300, X_test, y_test)

Training a KNeighborsClassifier using a training set size of 300. . .
Trained model in 0.0076 seconds
Made predictions in 0.0456 seconds.
F1 score for training set: 0.8722.
Made predictions in 0.0116 seconds.
F1 score for test set: 0.7482.


In [21]:
train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)

Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0055 seconds
Made predictions in 0.0016 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0021 seconds.
F1 score for test set: 0.6942.


In [22]:
train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)

Training a DecisionTreeClassifier using a training set size of 200. . .
Trained model in 0.0128 seconds
Made predictions in 0.0014 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0012 seconds.
F1 score for test set: 0.7132.


In [23]:
train_predict(clf_C, X_train_300, y_train_300, X_test, y_test)

Training a DecisionTreeClassifier using a training set size of 300. . .
Trained model in 0.0103 seconds
Made predictions in 0.0015 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.7167.


### Tabular Results
Here we record our results from above in the tables below.  
*You can learn how a table can be designed in Markdown [here](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). *

** Classifer 1 - LogisticRegression**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |       0.0018            | 0.0003                 |  0.8571          |  0.7612         |
| 200               |       0.0023            | 0.0002                 |  0.8380          |  0.7794         |
| 300               |       0.0032            | 0.0002                 |  0.8381          |  0.7910         |

** Classifer 2 - K-Nearest NeighBors**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |      0.0008             |  0.0019                | 0.7972           |  0.7068         |
| 200               |      0.0009             |  0.0022                | 0.8571           |  0.7121         |
| 300               |      0.0010             |  0.0031                | 0.8722           |  0.7482         |

** Classifer 3 - Decision Tree Classifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |      0.0013             |  0.0002                | 1.0000           |  0.6942         |
| 200               |      0.0019             |  0.0002                | 1.0000           |  0.7132         |
| 300               |      0.0033             |  0.0002                | 1.0000           |  0.7167         |

## Choosing the Best Model
In this final section, we choose from the three supervised learning models the *best* model to use on the student data. We then perform a grid search optimization for the chosen model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

*Based on the experiments we performed earlier and as we can see from the tables above, the Decision Tree Classifier is the algorithm with the lowest F1 Score on the testing set hence the worst model. Even though it is acheiving a perfect score of 1 on the training data, the score that ultimately matters is the one on the testing set since this tells us the ability of the model to generalize. The Decision Tree Classifier does not generalize well and that fact is confirmed here by our little experiment.    
As for the K-Nearest Neighbors algorithm, even though it seems to be doing well on the testing set, its prediction time is the worst of the three algorithms. Although its Training times are the best ones, for this particular situation, it seems reasonable to assume our algorithm is going to be used much more often for prediction than for testing.  
The model I will be choosing as the best model will then be the **Logistic Regression model** which not only has the highest F1 scores on the testing data but also a much lower Prediction Time than the k-nearest neighbors algorithm.*


### The Model in Layman's Terms

*Logistic Regression takes features about previous students (eg, their age, gender, etc) and creates a model that assigns a "weight" to these student features, that "weight" corresponds to the amount of information each feature is providing to the task of predicting the outcome. When we want to predict an outcome for a new student , we take the new student's features and combine them with the feature weights to the original students from which the model has been created. A final summed up value is applied to a function (called a "sigmoid") that then predicts the probability that a student will need an intervention or not.*  
 <img src="./2.png">  

### Implementation: Model Tuning
Here we fine tune the chosen model using grid search (`GridSearchCV`). We use the entire training set for this. In the code cell below, we implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters to be tuned for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the appropriate classifier and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [24]:
# Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer

# Create the list of parameters to tune
parameters = {'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000]}

# Initialize the classifier
clf = LogisticRegression(random_state = 0)

# Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label = 'yes')

# Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(estimator=clf, param_grid=parameters, scoring=f1_scorer)

# Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train,y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print ("Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train)))
print ("Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test)))

Made predictions in 0.0007 seconds.
Tuned model has a training F1 score of 0.8135.
Made predictions in 0.0006 seconds.
Tuned model has a testing F1 score of 0.7632.


In [15]:
grid_obj.best_params_

{'C': 0.001}

### Final F<sub>1</sub> Score
*After tuning C with 7 different values, the best one to use is 0.001 which is different from the default value 1. Accuracy however did not improve which is not surprising given than the classes are slightly skewed with more yes's than no's*.

## Conclusion
The landscape of education will continue to evolve over the coming years and machine learning will be at the core of future educational technology. A personal passion of mine is to build systems that adapt to the student's learning style and assess performance in a more nuanced manner than the alphanumerical measures of worth that are used today.